# 2a processing
- this code downloads and merges nldas for growing degree days, extreme degree days calculation

## downloading data

### Packages

In [89]:
import requests
import re
import os
import numpy as np
import datetime
import xarray
import dask

### Functions

In [86]:
def downloadData(url, output_path):
    '''
    Given a URL, assuming that we are logged into a session, 
    this function downloads data from NASA Earthdata and saves it to output_path.
    
    inputs:
      url: a string representing the file URL.
      output_path: the directory where the file should be saved.
    
    returns: nothing
    '''
    response = session.get(url, auth=(username, password), stream=True)
    if response.status_code == 200:
        # Try to get the filename from the Content-Disposition header
        cd = response.headers.get("content-disposition")
        if cd:
            fname_match = re.findall('filename="?([^";]+)"?', cd)
            if fname_match:
                filename = fname_match[0]
            else:
                filename = url.split("/")[-1]
        else:
            filename = url.split("/")[-1]
            
        # Write the content to a file in chunks
        with open(output_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:  # filter out keep-alive chunks
                    f.write(chunk)
        
def singleYearUrl(year):
    '''
    given a year, returns all possible urls for the year
    inputs year
    outputs
    # could and should be linked with download after every value in while loop. mkdir for each year
    '''
    tmp_downloads_dir = f"{output_dir}/{year}"
    
    try:
        os.mkdir(f"{tmp_downloads_dir}")
    except Exception:
        pass
        
    start_dt = datetime.datetime(year, 1, 1, 0, 0)
    end_dt = datetime.datetime(year, 12, 31, 23, 0)
    
    current_dt = start_dt
    while current_dt <= end_dt:
        julian_day = current_dt.strftime("%j")
        yyyymmdd = current_dt.strftime("%Y%m%d")
        # Get hour and minute as HHMM (e.g., "0000", "0100", etc.)
        hour_str = current_dt.strftime("%H%M")
        
        url = (f"{base_url}/{year}/{julian_day}/"
               f"NLDAS_FORA0125_H.A{yyyymmdd}.{hour_str}.020.nc")
    
        current_dt += datetime.timedelta(hours=1)
        downloadData(url, f"{tmp_downloads_dir}/NLDAS_FORA0125_H.A{yyyymmdd}.{hour_str}.020.nc")

### Inputs

In [87]:
# inputs 
output_dir = "/storage/home/cta5244/work/pyWBM_yield_data/NCEPNARR_NLDAS_Hist_Temp/tmp_downloads"
base_url = "https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_FORA0125_H.2.0"
start_year = 1979
end_year = 2026
username = os.environ.get("earthnasa_user")
password = os.environ.get("earthnasa_pass")

session = requests.Session()

### dask implementation

In [92]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="20GiB",
    walltime="03:00:00",
)

cluster.scale(jobs=30) 

2025-02-21 17:55:19,723 - distributed.core - INFO - Connection to tcp://10.6.8.47:51452 has been closed.
2025-02-21 17:55:19,724 - distributed.scheduler - INFO - Remove worker addr: tcp://10.6.8.47:41693 name: SLURMCluster-12 (stimulus_id='handle-worker-cleanup-1740178519.7241838')
2025-02-21 17:55:19,727 - distributed.core - INFO - Connection to tcp://10.6.8.47:51438 has been closed.
2025-02-21 17:55:19,727 - distributed.scheduler - INFO - Remove worker addr: tcp://10.6.8.47:34745 name: SLURMCluster-23 (stimulus_id='handle-worker-cleanup-1740178519.7278957')
2025-02-21 17:55:19,728 - distributed.batched - INFO - Batched Comm Closed <TCP (closed) Scheduler connection to worker local=tcp://146.186.150.13:46555 remote=tcp://10.6.8.47:51438>
Traceback (most recent call last):
  File "/storage/home/cta5244/mambaforge/envs/pyWBM/lib/python3.10/site-packages/distributed/comm/tcp.py", line 298, in write
    raise StreamClosedError()
tornado.iostream.StreamClosedError: Stream is closed

The ab

In [99]:
from dask.distributed import Client

client = Client(cluster)


/storage/home/cta5244/mambaforge/envs/pyWBM/lib/python3.10/site-packages/distributed/client.py:1612: VersionMismatchWarning: Mismatched versions found

+---------+----------+-----------+----------+
| Package | Client   | Scheduler | Workers  |
+---------+----------+-----------+----------+
| dask    | 2025.1.0 | 2025.1.0  | 2025.2.0 |
+---------+----------+-----------+----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [101]:
client

<Client: 'tcp://146.186.150.13:46555' processes=30 threads=30, memory=600.00 GiB>

In [110]:
results = []
for year in np.arange(start_year, end_year, 1):
    out = dask.delayed(singleYearUrl)(year=year)
    results.append(out)
    
results = dask.compute(*results)

SSLError: None: Max retries exceeded with url: /data/NLDAS/NLDAS_FORA0125_H.2.0/2001/023/NLDAS_FORA0125_H.A20010123.1100.020.nc (Caused by None)